## Spatial dynamics

In [77]:
import numpy as np
import matplotlib.pyplot as plt
from lgca import get_lgca
from scipy.integrate import  solve_ivp
from scipy.special import erf
from math import sqrt
from scipy.ndimage import laplace, convolve1d

In [78]:
a_max = 1.
a_min = 0.
na = 401
alpha, dalpha = np.linspace(a_min, a_max, num=na, retstep=True)
r_d = 0.01
r_b = 0.2
p_mut = 1
var = 0.00001
mean_mut_effect = 0
tmax = 50000
Da = var / 2
dens0 = 1 - r_d / r_b
ts = np.linspace(0, tmax, num=101)
nx = 100
xmax = 100
x, dx = np.linspace(0, xmax, nx, retstep=True, endpoint=False)
da = dalpha
p_m = 1
D = 0.00001
K = 1e6
dt = 1
Tfill = xmax / (2 * np.sqrt(D* (1.-r_d)))
tshow = [2, 10, 40, 100]
print(Tfill)

15891.043154093204


In [79]:
# cancer params
r_d = 0.25
r_b = 0.5
# p_mut = 1
p_l = 1e-8
T_d = 700
# T_d = 1400
T_p = 5e6
# T_p = 1e7
p_d = T_d * p_l
p_p = T_p * p_l
p_mut = 1 - (1 - p_d) * (1 - p_p)
p_pos = T_d / (T_d + T_p)
s_p = 0.001 * r_b
s_d = 0.2 * r_b
tmax = 100000
# Da = var / 2
dens0 = 1 - r_d / r_b
# mean_mut_effect = p_pos * s_d - (1 - p_pos) * s_p
# var = p_pos * s_d**2 + (1 - p_pos) * s_p**2
# secondmommu = var + mean_mut_effect**2
Ncrit = T_p * (s_p / r_b) / T_d / (s_d/r_b)**2
mucrit = s_d / T_p / r_b
K = np.ceil(Ncrit)
vtarget = 1./tmax * 100
gamma = -np.log(vtarget**2 / (4 * (r_b - r_d)))
D = 1. / (2 + np.exp(gamma))
Tfill = xmax / (2 * np.sqrt(D * (r_b - r_d)))
print(Tfill)

100000.09999994998


In [87]:
%matplotlib qt

In [80]:
l = 10
dims= l,
nodes = np.zeros((l, 3), dtype=int)
nodes[0, -1] = dens0 * K
lgca = get_lgca(ib=True, nodes=nodes, bc='refl', interaction='birthdeath_cancerdfe', ve=False, capacity=K,
                r_d=r_d, r_b=r_b, a_max=a_max, geometry='lin', dims=dims, restchannels=1, s_d=s_d, p_d=p_d, p_p=p_p, s_p=s_p, gamma=gamma)
lgca.timeevo(timesteps=tmax, record=True)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [90]:
skip = 200

In [88]:
x = np.arange(tmax+1)[::skip] / 4
plt.plot(x, lgca.dens_t.sum(-1)[::skip])
plt.plot(x, lgca.dens_t[::skip])
Nmax = K * (1 - r_d) * l
plt.plot([0, tmax/4], [Ncrit, Ncrit], 'k--', label='$N^*$')
plt.plot([0, tmax/4], [Nmax, Nmax], 'k:', label=r'$N_{\max}$')

In [91]:
lgca.plot_density(density_t=lgca.dens_t[::skip])

In [92]:
lgca.plot_prop_timecourse(nodes_t=lgca.nodes_t[::skip])

In [95]:
print(np.sum(lgca.nodes_t.astype(bool)))

In [93]:
lgca.plot_prop_spatial(nodes_t=lgca.nodes_t[::skip])